In [1]:
import os
from dotenv import load_dotenv
from autogen import ConversableAgent, config_list_from_json
from agentneo import AgentNeo, Tracer, Execution, launch_dashboard

# Load environment variables
load_dotenv()

# Initialize AgentNeo session
neo_session = AgentNeo(session_name="autogen_testing")
try:
    neo_session.create_project(project_name="calculator")
except:
    neo_session.connect_project(project_name="calculator")

# Initialize Tracer
tracer = Tracer(session=neo_session)
tracer.start()

# Define the calculator function
@tracer.trace_tool(name='calculator')
def calculator(a: int, b: int, operator: str) -> int:
    if operator == "+":
        return a + b
    elif operator == "-":
        return a - b
    elif operator == "*":
        return a * b
    elif operator == "/":
        return int(a / b)
    else:
        raise ValueError("Invalid operator")

# OpenAI configuration
config_list = [
    {
        "model": "gpt-4o-mini",
        "api_key": os.getenv("OPENAI_API_KEY")
    }
]


# Create the assistant agent
@tracer.trace_agent(name='AssistantAgent')
def create_assistant():
    return ConversableAgent(
        name="Assistant",
        system_message="You are a helpful AI assistant. "
        "You can help with simple calculations. "
        "Return 'TERMINATE' when the task is done.",
        llm_config={"config_list": config_list},
    )

# Create the user proxy agent
@tracer.trace_agent(name='UserProxyAgent')
def create_user_proxy():
    return ConversableAgent(
        name="User",
        llm_config=False,
        is_termination_msg=lambda msg: msg.get("content") is not None and "TERMINATE" in msg["content"],
        human_input_mode="NEVER",
    )

# Main function to run the conversation
# @tracer.trace_function(name='run_conversation')
@tracer.trace_agent(name='run_conversation')
def run_conversation():
    assistant = create_assistant()
    user_proxy = create_user_proxy()

    assistant.register_for_llm(name="calculator", description="A simple calculator")(calculator)
    user_proxy.register_for_execution(name="calculator")(calculator)

    # Start the conversation
    user_proxy.initiate_chat(assistant, message="What is (1423 - 123) / 3 + (32 + 23) * 5?")

# Run the conversation
run_conversation()

# Stop tracing
tracer.stop()


flaml.automl is not available. Please install flaml[automl] to enable AutoML functionalities.
DEBUG:agentneo.tracing.agent_tracer:Successfully updated and committed AgentCallModel with id 3
DEBUG:agentneo.tracing.agent_tracer:Successfully updated and committed AgentCallModel with id 4


Project 'calculator' found.
Tracing Started.
User (to Assistant):

What is (1423 - 123) / 3 + (32 + 23) * 5?

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
Assistant (to User):

***** Suggested tool call (call_skhTSAyM0KA8gqbEjSfM9rSs): calculator *****
Arguments: 
{"a": 1423, "b": 123, "operator": "-"}
***************************************************************************
***** Suggested tool call (call_PiyI9O66VZmhPWINDUq79lvA): calculator *****
Arguments: 
{"a": 32, "b": 23, "operator": "+"}
***************************************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION calculator...

>>>>>>>> EXECUTING FUNCTION calculator...
User (to Assistant):

User (to Assistant):

***** Response from calling tool (call_skhTSAyM0KA8gqbEjSfM9rSs) *****
1300
******************************************************************

DEBUG:agentneo.tracing.agent_tracer:Successfully updated and committed AgentCallModel with id 2


Tracing Completed.
Data saved to the database and JSON file.



In [2]:
# Execute metrics
from agentneo import Execution
exe = Execution(session=neo_session, trace_id=2)
exe.execute(metric_list=['goal_decomposition_efficiency', 'goal_fulfillment_rate', 'tool_correctness_metric', 'tool_call_success_rate_metric'])
metric_results = exe.get_results()

14:53:18 - LiteLLM:INFO: utils.py:2931 - 
LiteLLM completion() model= gpt-4o-mini; provider = openai
INFO:LiteLLM:
LiteLLM completion() model= gpt-4o-mini; provider = openai
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
14:53:20 - LiteLLM:INFO: utils.py:1008 - Wrapper: Completed Call, calling success_handler
INFO:LiteLLM:Wrapper: Completed Call, calling success_handler
14:53:20 - LiteLLM:INFO: utils.py:2931 - 
LiteLLM completion() model= gpt-4o-mini; provider = openai
INFO:LiteLLM:
LiteLLM completion() model= gpt-4o-mini; provider = openai
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
14:53:23 - LiteLLM:INFO: utils.py:1008 - Wrapper: Completed Call, calling success_handler
INFO:LiteLLM:Wrapper: Completed Call, calling success_handler
14:53:23 - LiteLLM:INFO: utils.py:2931 - 
LiteLLM completion() model= gpt-4o-mini; provider = openai
INFO:LiteLLM:
LiteLLM completion() model= gpt-4o-mini; provider =

In [3]:
metric_results

[{'metric_name': 'goal_decomposition_efficiency',
  'score': 1.0,
  'reason': 'The AI effectively decomposed the original goal into clear and manageable sub-tasks that directly correspond to the required calculations. Each sub-task was appropriately assigned to the calculator tool, and all calculations were executed successfully, achieving the overall user goal. The sub-tasks are logically sequenced, independent, and cover all necessary operations without redundancy, demonstrating an optimal decomposition process.',
  'result_detail': {'metric_name': 'goal_fulfillment_rate',
   'config': {},
   'result': {'originalGoal': 'Perform a series of mathematical calculations using a calculator function.',
    'subtasks': ['Subtract 123 from 1423.',
     'Add 32 and 23.',
     'Divide 1300 by 3.',
     'Multiply 55 by 5.',
     'Add 433 and 275.'],
    'score': 1.0,
    'reason': 'The AI effectively decomposed the original goal into clear and manageable sub-tasks that directly correspond to the

In [4]:
# Launch dashboard
launch_dashboard(port=3000)

INFO:root:Node.js version: v20.17.0
INFO:root:npm version: 10.8.2
INFO:root:Attempting to install React dependencies...
INFO:root:Dashboard output: 



up to date, audited 606 packages in 1s

168 packages are looking for funding
  run `npm fund` for details

found 0 vulnerabilities


INFO:root:Dashboard launched successfully. Access it at: http://localhost:3000
